In [1]:
import cdsapi
import xarray as xr
import rioxarray as rxr
import numpy as np

import os
import datetime
import requests

# Suppress warnings
requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning
)

In [2]:
# API Key
os.environ['URL'] = 'https://ads.atmosphere.copernicus.eu/api/v2'
os.environ['KEY'] = '<uid>:<key>'

# Product and Variable name
os.environ['PRODUCT'] = 'cams-global-atmospheric-composition-forecasts'
os.environ['VARIABLE'] = 'particulate_matter_2.5um'

# Area of Interest
os.environ['WEST'] = '90.0'
os.environ['EAST'] = '150.0'
os.environ['NORTH'] = '15.0'
os.environ['SOUTH'] = '-15.0'

In [3]:
url = os.environ.get('URL')
key = os.environ.get('KEY')
client = cdsapi.Client(url=url, key=key)

In [4]:
def download_latest_data(variable, forecast_hour):
    now = datetime.datetime.utcnow()

    # Calculate nearest model initialization
    mod = now - datetime.timedelta(hours = 10)
    obs = mod.date()
    if mod.hour > 12:
        base = datetime.datetime(
            obs.year, obs.month, obs.day, 12
        )
        time = '12:00'
    else:
        base = datetime.datetime(
            obs.year, obs.month, obs.day, 0
        )
        time = '00:00'
    
    print('Using forecast model initialized at', base.isoformat())

    # Observed Datetime
    obs_time = base + datetime.timedelta(hours = forecast_hour)
    
    # Area of Interest
    bbox = [
        float(os.environ.get('SOUTH')),
        float(os.environ.get('WEST')),
        float(os.environ.get('NORTH')),
        float(os.environ.get('EAST'))
    ]

    request = {
        'date' : f'{obs.isoformat()}/{obs.isoformat()}',
        'variable' : variable,
        'time' : time,
        'leadtime_hour' : str(forecast_hour),
        'area' : bbox,
        'type' : 'forecast',
        'format' : 'netcdf',
    }

    fname = f'{obs_time.strftime("%Y%m%dT%H")}.nc'
    product = os.environ.get('PRODUCT')
    client.retrieve(product, request, fname)

    return fname

In [5]:
variable = os.environ.get('VARIABLE')
fname = download_latest_data(variable, forecast_hour = 22)
ds = xr.open_dataset(fname).isel(time=0)

Using forecast model initialized at 2021-07-20T12:00:00


2021-07-21 15:10:04,231 INFO Welcome to the CDS
2021-07-21 15:10:04,233 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-atmospheric-composition-forecasts
2021-07-21 15:10:04,950 INFO Request is queued
2021-07-21 15:10:06,383 INFO Request is running
2021-07-21 15:10:08,207 INFO Request is completed
2021-07-21 15:10:08,210 INFO Downloading https://download-0001.copernicus-atmosphere.eu/cache-compute-0001/cache/data9/adaptor.mars_constrained.internal-1626855005.9631948-30100-14-dcd932dc-be45-4ae5-963c-f03598087e47.nc to 20210721T10.nc (799.2K)
2021-07-21 15:10:13,867 INFO Download rate 141.3K/s


In [6]:
variables = list(ds.data_vars)

for variable in variables:
    data = ds[variable] * 1e9

    date_string = os.path.splitext(fname)[0]
    out_fname = f'{variable}_{date_string}.tif'
    data.rio.set_spatial_dims('longitude', 'latitude', inplace=True)
    data.rio.set_crs(4326, inplace=True)
    data.rio.to_raster(out_fname)

    os.system(f'rio cogeo create {out_fname} {out_fname}')

/home/josef/miniconda3/envs/geo/lib/python3.9/site-packages/rio_cogeo/cogeo.py:189: IncompatibleBlockRasterSize: Block Size are bigger than raster sizes. Setting blocksize to 256
  warnings.warn(
Reading input: /home/josef/Documents/Git/GeospatialPython/CAMS/pm2p5_20210721T10.tif
Adding overviews...
Updating dataset tags...
Writing output to: /home/josef/Documents/Git/GeospatialPython/CAMS/pm2p5_20210721T10.tif
